### 練習問題2.3
平⾯上の⻑⽅形の表現を実装せよ  
(ヒント：練習問題 2.2を利⽤するといいかもしれない)。  
そのコンストラクタとセレクタを使って、  
ある⻑⽅形の外周の⻑さと⾯積を計算する⼿続きを書け。  
次に、⻑⽅形の異なる表現を実装せよ。  
適切な抽象化の壁を使ってシステムを設計し、  
外周と⾯積を求める同じ⼿続きがどちらの表現によっても動くようにできるだろうか。

<img src="2.03.png" width="50%">

<div style="text-align: center;">回答の条件</div>

In [1]:
; 点のコンストラクタ
(define (make-point x y)(cons x y))
; セレクタ
(define (x-point p)(car p))
(define (y-point p)(cdr p))
(define (print-point p)
    (newline)
    (display "(")
    (display (x-point p))
    (display ",")
    (display (y-point p))
    (display ")"))

; 線分コンストラクタ
(define (make-segment p1 p2)(cons p1 p2))
; セレクタ
(define (start-segment s)(car s))
(define (end-segment s)(cdr s))

; 中点を返す
(define (midpoint-segment s)
  (let ((p1 (start-segment s))
        (p2 (end-segment s)))
    (let ((x1 (x-point p1))
          (y1 (y-point p1))
          (x2 (x-point p2))
          (y2 (y-point p2)))
      (make-point (+ (/ (- x2 x1) 2) x1) (+ (/ (- y2 y1) 2) y1))
      )
    )
  )

In [2]:
; ⻑⽅形コンストラクタ
; 与えられた線分を対角線とした長方形とする。
(define (make-rect p1 p2)(make-segment p1 p2))

; 長方形セレクタ
(define (point-rect pos r)
  (let ((p1 (start-segment r))
        (p2 (end-segment r)))
    (let ((x1 (x-point p1))
          (y1 (y-point p1))
          (x2 (x-point p2))
          (y2 (y-point p2)))
        (
         cond ((= pos 0) (make-point (min x1 x2) (max y1 y2)))  ; top-left
              ((= pos 1) (make-point (max x1 x2) (max y1 y2)))  ; top-right
              ((= pos 2) (make-point (max x1 x2) (min y1 y2)))  ; bottom-right
              ((= pos 3) (make-point (min x1 x2) (min y1 y2)))  ; bottom-left
              (else (error "pos error." ()))
         )
      )
    )
  )

; 長方形の幅を返す
(define (width-rect r)
  (let ((p1 (point-rect 0 r))
        (p2 (point-rect 1 r)))
     (- (x-point p2) (x-point p1))
    )
  )

; 長方形の高さを返す
(define (height-rect r)
  (let ((p1 (point-rect 3 r))
        (p2 (point-rect 0 r)))
     (- (y-point p2) (y-point p1))
    )
  )

; 面積
(define (area-rect r)
  (let ((width (width-rect r))
        (height (height-rect r)))
        (* width height)
    )
  )
; 外周
(define (perimeter-rect r)
  (let ((width (width-rect r))
        (height (height-rect r)))
        (+ (* width 2) (* height 2))
    )
  )



(define p1 (make-point 3 3))
(print-point p1)
(define p2 (make-point 11 5))
(print-point p2)
(define p3 (midpoint-segment (make-segment p1 p2)))
(print-point p3)

(newline)
(define r (make-rect p1 p2))
(display (width-rect r))
(newline)
(display (height-rect r))
(newline)
(display (area-rect r))
(newline)
(display (perimeter-rect r))
(newline)


(3,3)
(11,5)
(7,4)
8
2
16
20


In [3]:
; 与えられた線分の、x軸に射影したときの幅を返す。
(define (segment-width s)
  (let ((p1 (start-segment s))
        (p2 (end-segment s)))
    (let ((x1 (x-point p1))
          (x2 (x-point p2)))
          (abs (- x1 x2))
      )
    )
  )
; 与えられた線分の、y軸に射影したときの幅(高さ)を返す。
(define (segment-height s)
  (let ((p1 (start-segment s))
        (p2 (end-segment s)))
    (let ((y1 (y-point p1))
          (y2 (y-point p2)))
          (abs (- y1 y2))
      )
    )
  )

; ⻑⽅形コンストラクタ
; 与えられた線分を対角線とした長方形
; 内部では中心点、幅、高さを記憶する。
(define (make-rect p1 p2)
  (let ((s (make-segment p1 p2)))
        (let ((midpoint (midpoint-segment s))
              (width (segment-width s))
              (height (segment-height s)))
          (cons midpoint (cons width height))
        )
    )
  )

; 長方形の幅を返す
(define (width-rect r)
  (car (cdr r))
  )

; 長方形の高さを返す
(define (height-rect r)
  (cdr (cdr r))
  )

(define r (make-rect p1 p2))
(display (width-rect r))
(newline)
(display (height-rect r))
(newline)
(display (area-rect r))
(newline)
(display (perimeter-rect r))
(newline)

8
2
16
20


In [4]:
; 別回答(コンストラクタのインターフェースが変わってしまっているので解答に適さない)
; コンストラクタ
; 内部は対角線として記憶する。
(define (make-rect center width height)
    (let ((half_width (/ width 2))
          (half_height (/ height 2)))
      (let ((x1 (- (x-point center) half_width))
            (y1 (- (y-point center) half_height))
            (x2 (+ (x-point center) half_width))
            (y2 (+ (y-point center) half_height)))
        (let ((p1 (make-point x1 y1))
              (p2 (make-point x2 y2)))
          (make-segment p1 p2)
        )
      )
    )
  )
; 長方形の幅を返す(元のセレクタと同じ定義)
(define (width-rect r)
  (let ((p1 (point-rect 0 r))
        (p2 (point-rect 1 r)))
     (- (x-point p2) (x-point p1))
    )
  )

; 長方形の高さを返す(元のセレクタと同じ定義)
(define (height-rect r)
  (let ((p1 (point-rect 3 r))
        (p2 (point-rect 0 r)))
     (- (y-point p2) (y-point p1))
    )
  )

(define p1 (make-point 3 3))
(print-point p1)
(define p2 (make-point 11 5))
(print-point p2)

(newline)
(define r (make-rect (make-point 7 4) 8 2))
(display (width-rect r))
(newline)
(display (height-rect r))
(newline)
(display (area-rect r))
(newline)
(display (perimeter-rect r))
(newline)


(3,3)
(11,5)
8
2
16
20
